In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.4 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gc

In [3]:
MIXTRAL = "mistralai/Mistral-7B-Instruct-v0.2"

In [4]:
messages = [
    {"role": "system", "content": "You are a skilled machine learning expert who helps machine learning engineers create perfectly curated datasets of any kind"},
    {"role": "user", "content": "Generate 30 labeled email messages as a JSON list. Each item must be a dictionary with 'text' and 'label', where label is either 'spam' or 'ham'."}
]


In [5]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MIXTRAL)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
model = AutoModelForCausalLM.from_pretrained(MIXTRAL, device_map="auto", quantization_config=quant_config)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [8]:
outputs = model.generate(inputs, max_new_tokens=250)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> [INST] You are a skilled machine learning expert who helps machine learning engineers create perfectly curated datasets of any kind

Generate 30 labeled email messages as a JSON list. Each item must be a dictionary with 'text' and 'label', where label is either 'spam' or 'ham'. [/INST] I'm an AI language model and not able to directly generate JSON lists or email messages. However, I can certainly help you create a JSON-like list of dictionaries representing 30 labeled email messages. Here's an example:

```json
[
  {
    "text": "Congratulations! You've won a free vacation to Bali! Click here to claim your prize.",
    "label": "spam"
  },
  {
    "text": "Hi John, just wanted to follow up on our meeting last week. I've attached the presentation slides for your review.",
    "label": "ham"
  },
  {
    "text": "10% off your next purchase! Use code SAVE10 at checkout.",
    "label": "spam"
  },
  {
    "text": "Dear [Name], your account balance is low. Please log in to add funds.",

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import re
import json
import csv
from io import BytesIO

# Model name
MIXTRAL = "mistralai/Mistral-7B-Instruct-v0.2"

# Quantization setup
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MIXTRAL)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MIXTRAL,
    device_map="auto",
    quantization_config=quant_config
)

# System and user prompt
messages = [
    {"role": "system", "content": "You are a skilled machine learning expert who helps machine learning engineers create perfectly curated datasets of any kind"},
    {"role": "user", "content": "Generate 50 labeled email messages as a JSON list. Each item must be a dictionary with 'text' and 'label', where label is either 'spam' or 'ham'. Do not explain."}
]

# Tokenize prompt
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

# Generate output
outputs = model.generate(
    inputs,
    max_new_tokens=1600,
    pad_token_id=tokenizer.eos_token_id
)

# Decode the model output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract JSON block using regex
match = re.search(r"\[\s*{.*?}\s*\]", decoded_output, re.DOTALL)
if match:
    json_str = match.group(0)
    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        print("Initial parse failed, cleaning and retrying...")
        json_str = json_str.replace("\n", " ").replace("\t", "")
        data = json.loads(json_str)
else:
    print("No valid JSON found.")
    data = []

# Save to CSV
if data:
    with open("emails_datasets1.csv", "w", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["text", "label"])
        writer.writeheader()
        writer.writerows(data)

    # Save to text file
    with open("emails_datasets1.txt", "w", encoding="utf-8") as f:
        for entry in data:
            f.write(f"{entry['label'].upper()}: {entry['text']}\n")

    print("✅ Dataset saved to emails_dataset.csv and emails_dataset.txt")
else:
    print("⚠️ No data to save.")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Dataset saved to emails_dataset.csv and emails_dataset.txt


In [10]:
pip install gradio transformers accelerate bitsandbytes


In [11]:
import gradio as gr
import torch
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Load Model
MIXTRAL = "mistralai/Mistral-7B-Instruct-v0.2"  # Replace if custom
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)
tokenizer = AutoTokenizer.from_pretrained(MIXTRAL)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MIXTRAL,
    device_map="auto",
    quantization_config=quant_config
)

import re
import json
import pandas as pd

def extract_json_block(text):
    """
    Extract a JSON array block from a string.
    Returns parsed JSON or None if extraction fails.
    """
    try:
        match = re.search(r'\[\s*{.*?}\s*\]', text, re.DOTALL)
        if not match:
            raise ValueError("No valid JSON array found in output.")
        json_str = match.group(0)
        return json.loads(json_str)
    except Exception as e:
        print("JSON extraction failed:", e)
        return None

def generate_dataset(prompt):
    # Build prompt for the LLM
    messages = [
        {"role": "system", "content": "You are a skilled machine learning expert who helps machine learning engineers create perfectly curated datasets of any kind."},
        {"role": "user", "content": f"{prompt}. Output a JSON list of dictionaries with keys 'text' and 'label'. Only return JSON."}
    ]

    # Tokenize input
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

    # Generate output
    outputs = model.generate(
        input_ids,
        max_new_tokens=1600,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract and parse JSON safely
    data = extract_json_block(decoded)
    if data is None:
        return "❌ Failed to extract valid JSON. Try rephrasing your request.", None, None

    # Save dataset
    df = pd.DataFrame(data)
    csv_path = "/tmp/generated_dataset.csv"
    txt_path = "/tmp/generated_dataset.txt"
    df.to_csv(csv_path, index=False)
    df.to_csv(txt_path, index=False, sep="\t")

    return "✅ Dataset generated successfully. Download below.", csv_path, txt_path


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
with gr.Blocks() as demo:
    gr.Markdown("#  Dataset Generator")

    with gr.Row():
        prompt_input = gr.Textbox(label="Describe your dataset", placeholder="e.g. Generate 50 spam and ham emails")
        run_button = gr.Button("Generate")

    output_text = gr.Textbox(label="Status")
    csv_file = gr.File(label="Download CSV")
    txt_file = gr.File(label="Download TXT")

    run_button.click(
        fn=generate_dataset,
        inputs=[prompt_input],
        outputs=[output_text, csv_file, txt_file]
    )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e137dc08f6cd72b424.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Here's a detailed line-by-line explanation of your code. This will help you understand every single part clearly, including why it's written that way and what it does:

🔹 Import Required Libraries
python
Copy
Edit
import gradio as gr
import torch
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
gradio: For creating a simple web-based interface so users can interact with the model via a browser.

torch: PyTorch, needed for model inference and GPU support.

json: To work with JSON data (important for parsing model outputs).

pandas: To structure the JSON as a DataFrame and save it in formats like CSV/TXT.

transformers: Hugging Face's library for loading and interacting with LLMs.

AutoTokenizer: Converts text to tokens.

AutoModelForCausalLM: Loads a causal language model.

BitsAndBytesConfig: Enables 4-bit quantization to reduce memory usage.

🔹 Model Setup
python
Copy
Edit
MIXTRAL = "mistralai/Mistral-7B-Instruct-v0.2"
Sets the model name — Mistral 7B, a powerful instruction-tuned language model from Hugging Face.

🔹 Quantization Config (4-bit)
python
Copy
Edit
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)
This configures the model to run efficiently in 4-bit mode, which reduces GPU memory usage. Explanation of params:

load_in_4bit=True: Enables loading in 4-bit.

bnb_4bit_use_double_quant=True: Applies a second quantization layer for better compression.

bnb_4bit_compute_dtype=torch.bfloat16: Sets computation precision to bfloat16 (faster on many GPUs).

bnb_4bit_quant_type="nf4": Uses NormalFloat4 quantization, optimal for LLMs.

🔹 Load Tokenizer
python
Copy
Edit
tokenizer = AutoTokenizer.from_pretrained(MIXTRAL)
tokenizer.pad_token = tokenizer.eos_token
Loads the tokenizer for the Mistral model.

Sets the pad_token to the eos_token (end-of-sequence) to avoid issues during generation (like warnings about attention masks).

🔹 Load the Model
python
Copy
Edit
model = AutoModelForCausalLM.from_pretrained(
    MIXTRAL,
    device_map="auto",
    quantization_config=quant_config
)
Loads the actual language model (CausalLM).

device_map="auto" lets Hugging Face figure out how to load it across CPU/GPU.

Applies the 4-bit quantization config.

📦 JSON Extraction Logic
🔹 Regex-Based JSON Extractor
python
Copy
Edit
def extract_json_block(text):
    """
    Extract a JSON array block from a string.
    Returns parsed JSON or None if extraction fails.
    """
    try:
        match = re.search(r'\[\s*{.*?}\s*\]', text, re.DOTALL)
        if not match:
            raise ValueError("No valid JSON array found in output.")
        json_str = match.group(0)
        return json.loads(json_str)
    except Exception as e:
        print("JSON extraction failed:", e)
        return None
Tries to extract a list of dictionaries ([{"text": ..., "label": ...}, {...}]) from model output using a regex.

re.DOTALL: Allows . to match newlines — helpful because LLM JSON often spans multiple lines.

If extraction fails, it returns None.

✨ Main Dataset Generator Function
python
Copy
Edit
def generate_dataset(prompt):
This function takes a prompt like:

"Generate 50 labeled email messages as spam or ham"

1. Build Prompt for the Model
python
Copy
Edit
messages = [
    {"role": "system", "content": "You are a skilled machine learning expert who helps machine learning engineers create perfectly curated datasets of any kind."},
    {"role": "user", "content": f"{prompt}. Output a JSON list of dictionaries with keys 'text' and 'label'. Only return JSON."}
]
Creates a structured prompt using a chat format:

System: Tells the model it is a dataset expert.

User: Provides the actual dataset request + forces it to respond in clean JSON only.

2. Tokenize for Input
python
Copy
Edit
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
Converts the chat prompt into tokens.

Returns a PyTorch tensor (return_tensors="pt") and sends it to GPU (to("cuda")).

3. Generate Response
python
Copy
Edit
outputs = model.generate(
    input_ids,
    max_new_tokens=1600,
    pad_token_id=tokenizer.eos_token_id
)
Asks the model to continue the prompt, limiting the output to 1600 tokens (to avoid cutting off mid-output).

Padding token is set to EOS token to avoid attention-related warnings.

4. Decode Text Output
python
Copy
Edit
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
Converts the generated tokens back to a readable string.


skip_special_tokens=True: Removes [INST], <s>,


etc.

5. Extract JSON Block
python
Copy
Edit
data = extract_json_block(decoded)
if data is None:
    return "❌ Failed to extract valid JSON. Try rephrasing your request.", None, None
Attempts to parse the JSON output.

If it fails, returns a message and None for the file paths.

6. Save to CSV and TXT
python
Copy
Edit
df = pd.DataFrame(data)
csv_path = "/tmp/generated_dataset.csv"
txt_path = "/tmp/generated_dataset.txt"
df.to_csv(csv_path, index=False)
df.to_csv(txt_path, index=False, sep="\t")
Converts the JSON list into a pandas DataFrame.

Saves it as:

generated_dataset.csv — comma-separated

generated_dataset.txt — tab-separated (good for manual inspection or other tools)

7. Return Success
python
Copy
Edit
return "✅ Dataset generated successfully. Download below.", csv_path, txt_path
Returns:

Success message

File paths for Gradio to display as download links

🧠 Summary of How This Works
Stage	Action
Prompting	Uses system + user messages to guide model
Tokenization	Converts the message into a model-understandable format
Generation	Gets a response from the Mistral LLM
Cleaning	Pulls out only the JSON block using regex
Saving	Exports as CSV and TXT for easy download

Let’s break this Gradio gr.Blocks() UI code line by line, so you fully understand how it creates an interactive app where users can request a labeled spam/ham email dataset and download it.

✅ GOAL
This UI lets users:

✍️ Input a text prompt like “Generate 50 spam and ham emails”

⚙️ Press a button to generate the dataset

📄 Download the result as CSV or TXT

🔧 with gr.Blocks() as demo:
python
Copy
Edit
with gr.Blocks() as demo:
gr.Blocks() is the Gradio layout system — it lets you build a flexible, custom web UI.

The UI you're about to define inside this block will be saved to a variable called demo, which you can .launch() later.

🧾 gr.Markdown(...)
python
Copy
Edit
gr.Markdown("# 📧 Spam/Ham Email Dataset Generator")
Adds a header to the app using Markdown.

The # means this is a level-1 header (big, bold title).

The 📧 emoji makes it visually engaging.

🧱 with gr.Row():
python
Copy
Edit
with gr.Row():
This creates a horizontal layout (row) so items inside it (like the textbox and button) appear side-by-side.

🧍 Input Box
python
Copy
Edit
prompt_input = gr.Textbox(label="Describe your dataset", placeholder="e.g. Generate 50 spam and ham emails")
This is a textbox where the user types what dataset they want.

label = visible title above the box

placeholder = text inside the box to give the user an example

▶️ Generate Button
python
Copy
Edit
run_button = gr.Button("Generate")
This is the action button users click to start the generation process.

You’ll later attach this button to a function using .click().

📤 Output Elements
python
Copy
Edit
output_text = gr.Textbox(label="Status")
csv_file = gr.File(label="Download CSV")
txt_file = gr.File(label="Download TXT")
Each of these is an output field:

output_text: A textbox where you'll show status (like ✅ success or ❌ fail).

csv_file: A download link for the CSV file.

txt_file: A download link for the TXT file.

These will be populated dynamically by your function.

🔁 Hooking Up Button → Function
python
Copy
Edit
run_button.click(
    fn=generate_dataset,
    inputs=[prompt_input],
    outputs=[output_text, csv_file, txt_file]
)
This links the button click to your function generate_dataset.

inputs=[prompt_input]: Sends the user’s textbox content into the function.

outputs=[output_text, csv_file, txt_file]: The function returns a tuple, and each item is sent to the corresponding UI component:

Status message → output_text

CSV file path → csv_file (which becomes a downloadable link)

TXT file path → txt_file (another link)

🚀 Launch the App
python
Copy
Edit
demo.launch()
This starts the Gradio app.

It will open in your browser (or show a link in notebooks/Colab).

You now have a fully functional dataset generation web app!

🔁 Recap of the Flow
Step	Action
User	Types in prompt
User	Clicks “Generate”
App	Sends prompt to generate_dataset()
LLM	Responds with JSON
App	Extracts JSON, saves CSV/TXT
App	Shows download links and a status message